# Assignment 1

## Exercise 1.1

rough sketch for ex 1.2:

1. define the „positive“ subspace P in the RGB cube
2. iterate over all pixels in I and check if in P or ~P
3. write result to new image
4. play around with size and shape of P and display binary image (**RESULT**)



In [5]:
from skimage import io, data, color, morphology
import numpy as np

image = io.imread('images/racecar/000.jpeg')
width, height, blub = image.shape
io.imshow(image)
mask = np.zeros_like(image[:,:,0])

for y in range(0,height):
    for x in range(0,width):
        if image[x,y,2] > 200:
            mask[x,y] = 255
            
mask2 = np.zeros_like(image[:,:,0])
io.imshow(mask2)

## Exercise 1.2

* starting from the binary color detection image
* erase noise with an erosion operation
* dilate once to get original size of object
* find connected components with one-pass algorithm
* extract bounding box on the fly
* draw bounding box on original image (**RESULT**)

## Exercise 1.3

* use your color detection and connected components algorithm
* implement simplest tracking algorithm
* draw history of all previous points on frame (**RESULT**)